In [1]:
# from flask_other.app import app
from flask import Flask
import os,sys
sys.path.append('../')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests
import uuid

##################################
url = 'http://localhost:8889/chatbotv1'


def get_data(sentence, uid):
    sessionId = cache.uid_dict[uid]
    response = requests.get(url, timeout=10, params={'data':sentence, 
                                                     'action':'chat', 
                                                     'sessionId':sessionId})
    
    if response.status_code == 200:
        req = response.json()
        if req.get('status').lower() == 'successful':
            msg = req.get('message')
        else:
            msg = req

        return msg
    else:
        return 'Internet Connection Issue, error code {}'.format(response.status_code)



@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )

@app.route("/createCookie", methods = ['POST'])
def createCookie():
    if request.method == 'POST':
        
        cookieid = request.form.get('cookie');
        if cookieid:
            print('already exists')
            print(cookieid)
            return('')
        else:
            print('new user')
            uid = str(uuid.uuid1())
            print('create cookie {}'.format(uid))
            return(str(uuid.uuid1()))



@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            query = request.form.get('query')#前端查询的内容
            uid = request.form.get('uid')
            print(uid)
            if query:

                    
                    # time.sleep(5)
                    result = get_data(query, uid)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('query duplicated! Use same id to save')
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                return str('请点击 new conversation')


    else:
        return 'ok'
    
    
@app.route("/newConversation", methods= ["POST"])
def new_conversation():
    
    print('new conversation called!!!!!!!!!!!!!!!!!!!!!!!')
    if request.method == 'POST':
        cookie = request.form.get('uid');
        if cookie:
            print(cookie)
            cache.new_conversation(cookie)
            return cookie
        else:
            print('cookie lost')
            return('error')

class Cache:
    def __init__(self):
#         self.new_conversation()
        self.uid_dict = {}
            
    def new_conversation(self):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                
                self.sessionId = msg['message']['sessionId']
                print(self.sessionId)
            else:
                
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.status_code)
            
    def new_conversation(self, uid):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                sessionId = msg['message']['sessionId']
                self.uid_dict[uid] = sessionId
                print(self.uid_dict[uid])
            else:
            
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.status_code)
            
        


        
cache = Cache()   




chatbotdb


In [2]:
app.debug = False
port = 6006
print('running at http://10.0.24.31:{}'.format(port))
app.run(host='0.0.0.0',port=port)    #这样用来监听所有的ip，团队调试用

 * Running on http://0.0.0.0:6006/ (Press CTRL+C to quit)


running at http://10.0.24.31:6006


10.0.25.20 - - [19/Jun/2018 20:24:36] "GET / HTTP/1.1" 200 -
10.0.25.20 - - [19/Jun/2018 20:24:36] "GET /static/jquery.min.js HTTP/1.1" 304 -
10.0.25.20 - - [19/Jun/2018 20:24:36] "GET /static/down.png HTTP/1.1" 304 -
10.0.25.20 - - [19/Jun/2018 20:24:46] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:24:46] "GET /static/avatar1.jpeg HTTP/1.1" 304 -
10.0.25.20 - - [19/Jun/2018 20:24:52] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:24:56] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:25:23] "GET / HTTP/1.1" 200 -
10.0.25.20 - - [19/Jun/2018 20:25:23] "POST /createCookie HTTP/1.1" 200 -


new user
create cookie e4e289de-73fe-11e8-b156-0242ac120002


10.0.25.20 - - [19/Jun/2018 20:25:26] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:25:26] "POST /predict HTTP/1.1" 200 -
10.0.25.20 - - [19/Jun/2018 20:25:26] "GET /static/avatar2.jpeg HTTP/1.1" 304 -


...

e4e2960e-73fe-11e8-b156-0242ac120002
saving to mongo db successfully!


10.0.25.20 - - [19/Jun/2018 20:30:00] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:30:00] "POST /predict HTTP/1.1" 200 -


e4e2960e-73fe-11e8-b156-0242ac120002
saving to mongo db successfully!


10.0.25.20 - - [19/Jun/2018 20:30:09] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.25.20 - - [19/Jun/2018 20:30:09] "POST /predict HTTP/1.1" 200 -


e4e2960e-73fe-11e8-b156-0242ac120002
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "a8605b09be6187e9825e90dbf8467a4a" }
query duplicated! Use same id to save
